<b><p style="text-align:center;font-size: 40px;"> Projet n°10</p><b>
<b><p style="text-align:center;font-size: 40px;"> Réalisez une application de recommandation de contenu</p><b>
<b><p style="text-align:center;font-size: 36px;"> Développement, entraînement et évalution de modèles de recommandation </p><b>

---

# Introduction

Ce notebook présente le développement et l’évaluation de plusieurs modèles de recommandation de contenu pour les articles du site "Globo.com". Après avoir préparé et nettoyé les données d’interactions utilisateurs-articles, nous mettons en œuvre trois approches principales : un modèle basé sur la popularité, un modèle de filtrage basé sur le contenu utilisant les embeddings d’articles, et un modèle de filtrage collaboratif avec une décomposition SVD. Chaque méthode est évaluée selon des métriques de précision et de rappel sur un même échantillon d’utilisateurs, afin de comparer leurs performances et d’illustrer les avantages de chaque approche dans le contexte de la recommandation personnalisée.

# Préparation des données

In [1]:
# Imports
import pandas as pd
import numpy as np
import os
import pickle

from surprise.model_selection import RandomizedSearchCV
from surprise import Dataset, Reader, SVDpp
from tqdm.notebook import tqdm

On commence par charger et concaténer toutes les données sur les clics utilisateurs :

In [ ]:
clicks_dir = 'news-portal-user-interactions-by-globocom/clicks'
n_hours = 385
clicks_list = []
for i in range(n_hours):
    path = os.path.join(clicks_dir, f'clicks_hour_{i:03d}.csv') 
    if os.path.exists(path):
        df = pd.read_csv(path)
        clicks_list.append(df)    
clicks = pd.concat(clicks_list, ignore_index=True)
clicks

,user_id,session_id,session_start,session_size,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,0,1506825423271737,1506825423000,2,157541,1506826828020,4,3,20,1,20,2
1,0,1506825423271737,1506825423000,2,68866,1506826858020,4,3,20,1,20,2
2,1,1506825426267738,1506825426000,2,235840,1506827017951,4,1,17,1,16,2
3,1,1506825426267738,1506825426000,2,96663,1506827047951,4,1,17,1,16,2
4,2,1506825435299739,1506825435000,2,119592,1506827090575,4,1,17,1,24,2
...,...,...,...,...,...,...,...,...,...,...,...,...
2988176,10051,1508211372158328,1508211372000,2,84911,1508211557302,4,3,2,1,25,1
2988177,322896,1508211376302329,1508211376000,2,30760,1508211672520,4,1,17,1,25,2
2988178,322896,1508211376302329,1508211376000,2,157507,1508211702520,4,1,17,1,25,2
2988179,123718,1508211379189330,1508211379000,2,234481,1508211513583,4,3,2,1,25,2


On récupère ensuite les ID des différents articles consultés et on charge les embeddings :

In [3]:
# Extraire les IDs des articles consultés
articles = clicks['click_article_id'].value_counts().index.to_numpy(dtype=int)

# Charger les embeddings complets
with open('news-portal-user-interactions-by-globocom/articles_embeddings.pickle', 'rb') as f:
    embeddings = pickle.load(f)

# normaliser les embeddings
embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

On isole ensuite un dataframe contenant les ID des utilisateurs et les numéros des articles sur lesquels ils ont cliqué. On attribue une "note" de 1 pour toutes ces interactions, qui servira pour le cas du modèle collaboratif :

In [4]:
# Crée le users_df avec rating implicite binaire (1 pour chaque clic)
users_df = clicks[['user_id', 'click_article_id']].copy()
users_df['rating'] = 1  # implicite binaire

users_df = users_df.sort_values(by=['user_id', 'click_article_id']).reset_index(drop=True)

display(users_df)

,user_id,click_article_id,rating
0,0,68866,1
1,0,87205,1
2,0,87224,1
3,0,96755,1
4,0,157541,1
...,...,...,...
2988176,322894,168401,1
2988177,322895,63746,1
2988178,322895,289197,1
2988179,322896,30760,1


On procède à la séparation en données de test et d'entrainement en utilisant une méthodologie de "leave_one_out" : pour chaque utilisateur, une seule de ses intéraction est placée dans les données de test, les autres sont regroupées dans les données d'entraînement.

In [5]:
def train_test_split_leave_one_out(users_df):
    train_rows = []
    test_rows = []

    grouped = users_df.groupby('user_id')

    for user_id, group in grouped:
        if len(group) < 2:
            continue  # on ne garde que les utilisateurs avec au moins 2 interactions
        group_sorted = group.sort_values('click_article_id')
        test_row = group_sorted.iloc[-1]
        train_rows.append(group_sorted.iloc[:-1])
        test_rows.append(test_row)

    train_df = pd.concat(train_rows)
    test_df = pd.DataFrame(test_rows)

    return train_df, test_df

In [6]:
train_df, test_df = train_test_split_leave_one_out(users_df)
print(f"Train set size: {len(train_df)}, Test set size: {len(test_df)}")

Train set size: 2665284, Test set size: 322897


On s'assure que les différents identifiants d'utilisateurs et d'articles sont bien des entiers :

In [7]:
train_df['user_id'] = train_df['user_id'].astype(int)
train_df['click_article_id'] = train_df['click_article_id'].astype(int)
test_df['user_id'] = test_df['user_id'].astype(int)
test_df['click_article_id'] = test_df['click_article_id'].astype(int)

Pour évaluer les performances des modèles, on ne pourra pas utiliser l'ensemble complet des données de test (temps de calcul trop long). On sélectionne donc un échantillion aléatoire de 10 0000 utilisateurs sur lesquels on évaluera les performances des modèles. Tous les modèles seront bien évalués sur ce meme "users_sample" :

In [8]:
n_users = 10000

random_state = 42
np.random.seed(random_state)

all_user_ids = test_df['user_id'].unique()
users_sample = np.random.choice(all_user_ids, size=n_users, replace=False)

# Baseline : modèle basé sur la popularité

Le premier modèle que l'on évalue est un modèle baseline basé sur la popularité : on recommande aux utilisateurs les articles qui ont été le plus consultés, c'est à dire ceux qui cumulent le plus de clics. On fait ces recommandations par ordre décroissant de popularité.

In [ ]:
def evaluate_popularity_model(train_df, test_df, users_sample, k=20):
    """
    Évalue un modèle de recommandation basé sur la popularité des articles.

    Pour chaque utilisateur de l'échantillon, recommande les k articles les plus populaires
    (non encore vus par l'utilisateur) et calcule la précision et le rappel en comparant
    avec les articles réellement consultés dans le jeu de test.

    Args:
        train_df (pd.DataFrame): Données d'entraînement contenant les interactions utilisateur-article.
        test_df (pd.DataFrame): Données de test contenant les interactions utilisateur-article.
        users_sample (array-like): Liste ou tableau des identifiants d'utilisateurs à évaluer.
        k (int, optional): Nombre d'articles à recommander par utilisateur. Par défaut à 20.

    Returns:
        tuple: (précision moyenne, rappel moyen) calculés sur l'échantillon d'utilisateurs.
    """
    # Popularité globale
    most_popular_articles = train_df['click_article_id'].value_counts().index.values

    # Pré-calcul user → articles vus / test
    user_seen = train_df.groupby("user_id")["click_article_id"].apply(set).to_dict()
    user_true = test_df.groupby("user_id")["click_article_id"].apply(set).to_dict()

    precisions, recalls = [], []

    for user_id in users_sample:
        if user_id not in user_true:  # pas de vérité terrain
            continue

        seen = user_seen.get(user_id, set())
        true_articles = user_true[user_id]
        if not true_articles:
            continue

        # Récupérer les top-k non vus
        recommended_ids = []
        for aid in most_popular_articles:
            if aid not in seen:
                recommended_ids.append(aid)
                if len(recommended_ids) == k:
                    break

        # Evaluation
        hits = len(set(recommended_ids) & true_articles)
        precisions.append(hits / k)
        recalls.append(hits / len(true_articles))

    if not precisions:
        return 0.0, 0.0

    return float(np.mean(precisions)), float(np.mean(recalls))

In [10]:
precision, recall = evaluate_popularity_model(
    train_df,
    test_df,
    users_sample,
    k=20
)
print("Résultats de l'évaluation du modèle de recommandation basé sur la popularité :\n")
print(f"Précision@20: {precision:.4f}")
print(f"Rappel@20: {recall:.4f}")

Résultats de l'évaluation du modèle de recommandation basé sur la popularité :

Précision@20: 0.0055
Rappel@20: 0.1107


# Filtrage basé sur le contenu

Le modèle de recommandation basé sur le contenu utilise les caractéristiques des articles (embeddings) pour proposer des contenus similaires à ceux déjà consultés par l’utilisateur. Pour chaque utilisateur, on construit un "profil utilisateur" en calculant la moyenne pondérée des embeddings des articles sur lesquels il a cliqué. Ensuite, on recommande les articles dont les embeddings sont les plus proches (similaires) de ce profil, en excluant ceux déjà vus. L’évaluation du modèle consiste à mesurer la précision et le rappel des recommandations sur un échantillon d’utilisateurs, en comparant les articles recommandés avec ceux réellement consultés dans les données de test. Cette approche permet de personnaliser les recommandations en fonction des préférences individuelles, même pour les utilisateurs ayant peu d’interactions.

In [3]:
def create_user_profile(user_id, users_df, embeddings):
    """
    Crée le profil d'un utilisateur sous forme de vecteur d'embedding moyen.

    Args:
        user_id (int): Identifiant de l'utilisateur.
        users_df (pd.DataFrame): DataFrame contenant les interactions utilisateur-article, 
                                 avec au moins les colonnes 'user_id', 'click_article_id', et 'rating'.
        embeddings (np.ndarray): Tableau numpy des embeddings d'articles, indexé par article_id.

    Returns:
        np.ndarray or None: Vecteur profil utilisateur normalisé (embedding moyen pondéré), 
                            ou None si l'utilisateur n'a aucune interaction.
    """
    user_data = users_df[users_df['user_id'] == user_id]
    if user_data.empty:
        return None

    vectors, weights = [], []
    for _, row in user_data.iterrows():
        article_id = int(row['click_article_id'])
        vectors.append(embeddings[article_id])
        weights.append(row['rating'])

    profile = np.average(vectors, axis=0, weights=weights)

    # Normalisation du profil
    norm = np.linalg.norm(profile)

    return profile / norm

In [ ]:
def get_recommendations_content_based(user_id, data, embeddings, article_ids, k=5):
    """
    Génère les recommandations d'articles pour un utilisateur donné en utilisant un modèle basé sur le contenu.

    Pour chaque utilisateur, le profil est construit comme la moyenne pondérée des embeddings des articles déjà consultés.
    Les articles non vus sont ensuite classés selon leur similarité cosinus avec le profil utilisateur, et les k plus similaires sont recommandés.

    Args:
        user_id (int): Identifiant de l'utilisateur.
        data (pd.DataFrame): DataFrame contenant les interactions utilisateur-article, avec au moins 'user_id' et 'click_article_id'.
        embeddings (np.ndarray): Tableau numpy des embeddings d'articles, indexé par article_id.
        article_ids (array-like): Liste ou tableau des identifiants d'articles disponibles pour la recommandation.
        k (int, optional): Nombre d'articles à recommander. Par défaut à 5.

    Returns:
        list: Liste des identifiants des k articles recommandés (non vus), triés par similarité décroissante.
    """
    profile = create_user_profile(user_id, data, embeddings)
    if profile is None:
        return []

    # Articles déjà cliqués par l'utilisateur
    seen_articles = set(data[data['user_id'] == user_id]['click_article_id'].unique())

    # Similarité entre profil et tous les articles
    similarity = embeddings @ profile

    # Restriction aux articles présents dans le dataset (et non cliqués)
    unseen_article_ids = [aid for aid in article_ids if aid not in seen_articles]
    if not unseen_article_ids:
        return []

    unseen_scores = similarity[unseen_article_ids]

    # Top-k avec argpartition
    top_k_idx = np.argpartition(unseen_scores, -k)[-k:]
    top_k_idx = top_k_idx[np.argsort(unseen_scores[top_k_idx])[::-1]]

    return np.array(unseen_article_ids)[top_k_idx].tolist()

In [ ]:
def evaluate_content_based(
    train_df,
    test_df,
    embeddings,
    article_ids, 
    users_sample,
    k=20
):
    """
    Évalue le modèle de recommandation basé sur le contenu.

    Pour chaque utilisateur de l'échantillon, recommande les k articles les plus similaires à son profil
    (non encore vus par l'utilisateur) et calcule la précision et le rappel en comparant
    avec les articles réellement consultés dans le jeu de test.

    Args:
        train_df (pd.DataFrame): Données d'entraînement contenant les interactions utilisateur-article.
        test_df (pd.DataFrame): Données de test contenant les interactions utilisateur-article.
        embeddings (np.ndarray): Tableau numpy des embeddings d'articles, indexé par article_id.
        article_ids (array-like): Liste ou tableau des identifiants d'articles disponibles pour la recommandation.
        users_sample (array-like): Liste ou tableau des identifiants d'utilisateurs à évaluer.
        k (int, optional): Nombre d'articles à recommander par utilisateur. Par défaut à 20.

    Returns:
        tuple: (précision moyenne, rappel moyen) calculés sur l'échantillon d'utilisateurs.
    """
    precisions = []
    recalls = []

    for user_id in tqdm(users_sample, desc="Évaluation des recommandations"):
        
        recommended_ids = get_recommendations_content_based(user_id, train_df, embeddings, article_ids, k=k)

        true_articles = test_df[test_df['user_id'] == user_id]['click_article_id'].unique()

        hits = np.isin(recommended_ids, true_articles).sum()

        precision = hits / len(recommended_ids)
        recall = hits / len(true_articles)

        precisions.append(precision)
        recalls.append(recall)

    if len(precisions) == 0:
        return 0.0, 0.0

    avg_precision = np.mean(precisions)
    avg_recall = np.mean(recalls)

    return avg_precision, avg_recall


In [14]:
precision, recall = evaluate_content_based(
    train_df,
    test_df,
    embeddings,
    articles,
    users_sample,
    k=20,
)
print("Résultats de l'évaluation du modèle de recommandation basé sur le contenu :\n")
print(f"Précision@20: {precision:.4f}")
print(f"Rappel@20: {recall:.4f}")

Évaluation des recommandations:   0%|          | 0/10000 [00:00<?, ?it/s]

Résultats de l'évaluation du modèle de recommandation basé sur le contenu :

Précision@20: 0.0014
Rappel@20: 0.0281


Les performances de ce modèle sont très mauvaises, avec un rappel largement inférieur au modèle de popularité baseline. Cela vient très certainement du fait que pour chaque utilisateur, les articles qu'il a consultés ont une faible similarité entre eux (voir l'EDA dans le notebook eda_globocom.ipynb).

# Filtrage collaboratif

Dans cette partie, nous mettons en œuvre un modèle de recommandation basé sur le filtrage collaboratif, plus précisément en utilisant la méthode SVD++ via la librairie Surprise. Le filtrage collaboratif exploite les interactions passées entre utilisateurs et articles pour prédire les préférences futures. 

Dans notre cas, c'est l'algorithme SVD++ qui a été utilisé à la place de l'algorithme SVD classique. En effet, la décomposition SVD++ prend en compte non seulement des notes explicites attribuées par les utilisateurs, mais aussi les interactions implicites, c’est-à-dire les articles sur lesquels l’utilisateur a cliqué, même sans donner de note. Dans notre cas, les données sont uniquement des clics (interactions implicites), et SVD++ permet d’exploiter ces informations pour mieux modéliser les préférences des utilisateurs. Cela améliore généralement la qualité des recommandations par rapport à une décomposition SVD classique, qui ne considère que des notes explicites.

Voici les étapes principales :
- Les données d’interactions (clics) sont transformées en une matrice utilisateur-article avec des notes implicites (1 pour chaque clic).
- Le modèle SVD++ est entraîné sur ces interactions pour apprendre des représentations latentes des utilisateurs et des articles.
- Pour chaque utilisateur, le modèle prédit les scores de tous les articles non encore vus et recommande ceux avec les meilleurs scores.
- Les recommandations sont ensuite évaluées en termes de précision et de rappel sur un échantillon d’utilisateurs, en comparant les articles recommandés avec ceux réellement consultés dans les données de test.

Cette approche permet de capturer des similarités entre utilisateurs et articles, sans utiliser d’informations sur le contenu des articles.

Création de la matrice d'interactions utilisateurs-articles :

In [15]:
reader = Reader(rating_scale=(0, 1))
data = Dataset.load_from_df(train_df[['user_id', 'click_article_id', 'rating']], reader)
trainset = data.build_full_trainset()

Entraînement du modèle (calcul de la décomposition SVD) :

In [16]:
svdpp = SVDpp(random_state=42, n_factors=1, n_epochs=5)
svdpp.fit(trainset)

Nous n'avons malheureusement pas réussi a effectué de recherche par grille (gridSearchCV) pour essayer d'optimiser les paramètres du modèle. Chaque exécution de l'algorithme RandomizedSearchCV de Suprise entraînant un crash du Kernel, nous avons essayé d'optimiser le modèle au mieux en exécutant plusieurs fois l'entrainement et l'évaluation avec différents jeux d'hyperparamètres.

Sauvegarde du modèle entrainé :

In [17]:
with open("model_svdpp.pkl", "wb") as f:
    pickle.dump(svdpp, f)

In [ ]:
def get_recommendations_collaborative(user_id, data, svd_model, article_ids, k=5):
    """
    Génère les recommandations d'articles pour un utilisateur donné en utilisant un modèle collaboratif SVD++.

    Pour chaque utilisateur, le modèle prédit les scores pour tous les articles non encore vus,
    puis recommande les k articles avec les meilleurs scores.

    Args:
        user_id (int): Identifiant de l'utilisateur.
        data (pd.DataFrame): DataFrame contenant les interactions utilisateur-article, avec au moins 'user_id' et 'click_article_id'.
        svd_model: Modèle SVD++ entraîné via la librairie Surprise.
        article_ids (array-like): Liste ou tableau des identifiants d'articles disponibles pour la recommandation.
        k (int, optional): Nombre d'articles à recommander. Par défaut à 5.

    Returns:
        list: Liste des identifiants des k articles recommandés (non vus), triés par score prédictif décroissant.
    """
    # Articles déjà vus
    seen_articles = set(data[data['user_id'] == user_id]['click_article_id'].unique())

    # Selection des articles non vus
    unseen_article_ids = [aid for aid in article_ids if aid not in seen_articles]
    if not unseen_article_ids:
        return []

    # Générer les prédictions
    predictions = []
    for aid in unseen_article_ids:
        try:
            pred = svd_model.predict(user_id, aid)
            predictions.append((aid, pred.est))
        except:
            continue  # utilisateur ou article inconnu du modèle

    # Trier et extraire top-k
    top_k = sorted(predictions, key=lambda x: x[1], reverse=True)[:k]
    recommended_ids = [aid for aid, _ in top_k]

    return recommended_ids

In [ ]:
def evaluate_collaborative(
    train_df,
    test_df,
    svd_model,
    article_ids, 
    users_sample,
    k=20
):
    """
    Évalue un modèle de recommandation collaboratif basé sur SVD++.

    Pour chaque utilisateur de l'échantillon, recommande les k articles non vus avec les meilleurs scores prédits
    par le modèle SVD++, puis calcule la précision et le rappel en comparant avec les articles réellement consultés
    dans le jeu de test.

    Args:
        train_df (pd.DataFrame): Données d'entraînement contenant les interactions utilisateur-article.
        test_df (pd.DataFrame): Données de test contenant les interactions utilisateur-article.
        svd_model: Modèle SVD++ entraîné via la librairie Surprise.
        article_ids (array-like): Liste ou tableau des identifiants d'articles disponibles pour la recommandation.
        users_sample (array-like): Liste ou tableau des identifiants d'utilisateurs à évaluer.
        k (int, optional): Nombre d'articles à recommander par utilisateur. Par défaut à 20.

    Returns:
        tuple: (précision moyenne, rappel moyen) calculés sur l'échantillon d'utilisateurs.
    """
    precisions = []
    recalls = []

    for user_id in tqdm(users_sample, desc="Évaluation des recommandations"):
        
        recommended_ids = get_recommendations_collaborative(user_id, train_df, svd_model, article_ids, k=k)

        true_articles = test_df[test_df['user_id'] == user_id]['click_article_id'].unique()

        hits = np.isin(recommended_ids, true_articles).sum()

        precision = hits / len(recommended_ids)
        recall = hits / len(true_articles)

        precisions.append(precision)
        recalls.append(recall)

    if len(precisions) == 0:
        return 0.0, 0.0

    avg_precision = np.mean(precisions)
    avg_recall = np.mean(recalls)

    return avg_precision, avg_recall

In [20]:
precision, recall = evaluate_collaborative(
    train_df,
    test_df,
    svdpp,
    articles,
    users_sample,
    k=20
)
print("Résultats de l'évaluation du modèle de recommandation collaboratif :\n")
print(f"Précision@20: {precision:.4f}")
print(f"Rappel@20: {recall:.4f}")

Évaluation des recommandations:   0%|          | 0/10000 [00:00<?, ?it/s]

Résultats de l'évaluation du modèle de recommandation collaboratif :

Précision@20: 0.0069
Rappel@20: 0.1389


Le filtrage collaboratif permet d'obtenir des performances légèrement meilleures que le modèle de popularité (rappel de 0.14 contre 0.11). 

# Conclusion

Parmi les trois approches testées, le modèle de filtrage collaboratif (SVD++) s’est révélé le plus performant, surpassant le modèle de popularité et surtout le modèle basé sur le contenu, qui affiche les résultats les plus faibles. Malgré cela, les performances globales restent modestes, même pour le collaboratif. Cela s’explique probablement par le manque de diversité dans les interactions : comme observé dans l’EDA, un tiers des utilisateurs n’ont cliqué que sur deux articles, et les données ne comportent pas de véritables notes ou préférences explicites, mais uniquement des identifiants d’articles consultés. Cette limitation réduit la capacité des modèles à bien capturer les préférences individuelles et à générer des recommandations pertinentes.

Dans le cadre du développement d'une application de recommandation de contenu, c'est donc le modèle avec filtrage collaboratif que nous avons décidé d'utiliser.